In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [2]:
dtype = torch.FloatTensor

In [3]:
sentence = [ "i like dog", "i love coffee", "i hate milk", "i like everything", "i hate cat"]

word_list = " ".join(sentence).split()
word_list = list(set(word_list))
word_dict = {w:i for i,w in enumerate(word_list)}
number_dict = {i:w for i,w in enumerate(word_list)}
n_class = len(word_list) # V

In [4]:
word_dict

{'milk': 0,
 'love': 1,
 'everything': 2,
 'like': 3,
 'cat': 4,
 'i': 5,
 'hate': 6,
 'coffee': 7,
 'dog': 8}

In [5]:
# NNLM 参数
n_step = 2 # n-1 in paper， 3-gram
n_hidden = 2 # h in paper,
m = 2 # m in paper，word 2 features也可以是30，60等等

In [6]:
# minibatch
def make_batch(sentence):
    input_batch = []
    target_batch = []
    
    for sen in sentence:
        word = sen.split()
        vector = [word_dict[w] for w in word[:-1]]
        target = word_dict[word[-1]]
        
        input_batch.append(vector)
        target_batch.append(target)
    return input_batch, target_batch

In [7]:
# Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)  # V*m
        # h*(n-1)m
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))
        # h*m
        self.W = nn.Parameter(torch.randn(n_step * m, n_class).type(dtype))
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))

    def forward(self, X):
        X = self.C(X)
        X = X.view(-1, n_step * m) # [batch_size, n_step * n_class]
        tanh = torch.tanh(self.d + torch.mm(X, self.H)) # [batch_size, n_hidden]
        output = self.b + torch.mm(X, self.W) + torch.mm(tanh, self.U) # [batch_size, n_class]
        return output

In [8]:
model = NNLM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
input_batch, target_batch = make_batch(sentence)
input_batch = Variable(torch.LongTensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

In [10]:
# Training
for epoch in range(5000):

    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.686435
Epoch: 2000 cost = 0.575771
Epoch: 3000 cost = 0.561702
Epoch: 4000 cost = 0.557673
Epoch: 5000 cost = 0.556073


In [22]:
# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]

In [23]:
predict

tensor([[2],
        [7],
        [0],
        [2],
        [0]])

In [29]:
for i in predict:
    
    print(number_dict[int(predict[i])])

milk


IndexError: index 7 is out of bounds for dimension 0 with size 5

In [27]:
# Test
print([sen.split()[:2] for sen in sentence], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['i', 'like'], ['i', 'hate']] -> ['everything', 'coffee', 'milk', 'everything', 'milk']
